In [107]:
# If needed:
# !pip install pandas numpy matplotlib openpyxl statsmodels

import re, warnings, os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from statsmodels.tsa.arima.model import ARIMA
warnings.filterwarnings("ignore")


In [ ]:
FILE_PATH  = r"D:\arima project\ckd stroke\URBANIZATION COMPLETE CKD & STROKE.xlsx"
SHEET_NAME = "data final"        # None to auto-pick; otherwise put exact sheet name
END_YEAR   = 2043
OUTPUT_DIR = Path(r"D:\arima project\code\ckd_stroke_results\urbanization")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Optional: if auto-detection picks the wrong AAMR column, set this to the exact column name:
MANUAL_AAMR_COL = None  # e.g., "Age Adjusted Rate"


In [109]:
plt.rcParams.update({
    "figure.figsize": (13, 7.6),
    "figure.dpi": 160,
    "axes.grid": True,
    "grid.alpha": 0.25,
    "font.size": 14,
    "font.weight": "bold",
    "axes.labelsize": 16,
    "axes.labelweight": "bold",
    "axes.titlesize": 20,
    "axes.titleweight": "bold",
    "legend.fontsize": 12,
    "legend.frameon": True,
    "legend.title_fontsize": 13,
    "lines.linewidth": 2.6,
    "lines.markersize": 6.5,
})
def style_axes(ax):
    ax.tick_params(axis="both", labelsize=14, width=1.9, length=6)
    for s in ax.spines.values():
        s.set_linewidth(2.0)


In [110]:
def _flatten_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Flatten multirow headers, normalize whitespace, and deduplicate names."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            " ".join([str(x) for x in tup if (str(x) != "nan" and str(x).strip() != "")]).strip()
            for tup in df.columns.to_list()
        ]
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]
    seen, newcols = {}, []
    for c in df.columns:
        if c in seen:
            seen[c] += 1; newcols.append(f"{c}.{seen[c]}")
        else:
            seen[c] = 0;  newcols.append(c)
    df.columns = newcols
    return df

def _pick_col(df, patterns, required=True, exclude_regex=None):
    low = {c: str(c).strip().lower() for c in df.columns}
    for pat in patterns:
        rx = re.compile(pat)
        for c, lc in low.items():
            if rx.search(lc):
                if exclude_regex and re.search(exclude_regex, lc):
                    continue
                return c
    if required:
        raise KeyError(f"Could not find any of: {patterns}")
    return None

def find_year(df):  # prefer 'Year'
    return _pick_col(df, [r"\byear\b", r"^yr$", r"\bcalendar\s*year\b"])

def find_aamr(df):
    if MANUAL_AAMR_COL and MANUAL_AAMR_COL in df.columns:
        return MANUAL_AAMR_COL
    # Be precise so we don't confuse with "Age group"
    pats = [
        r"\bage[-\s]*adjust(ed)?\s*rate\b",
        r"\baamr\b",
        r"\bage[-\s]*standard(ized)?\s*rate\b"
    ]
    # also allow generic "age adjust" + "rate" together
    low = {c: str(c).strip().lower() for c in df.columns}
    # exact patterns first
    for p in pats:
        try: return _pick_col(df, [p])
        except: pass
    # fallback: both 'age'+'adjust' and 'rate' in name
    for c, lc in low.items():
        if all(k in lc for k in ["age","adjust"]) and "rate" in lc:
            return c
    # last resort: a column literally named 'AAMR' somewhere
    return _pick_col(df, [r"\baamr\b"])

def find_group(df):
    """True grouping column (Age group / variable / Sex / Race / Region / Urbanization / State)."""
    exclude = r"age\s*adjust|aamr|rate|ci|confidence|se|std|mean|median|total|overall"
    patterns = [
        r"\bvariable\b",
        r"\bage\s*group(s)?\b|age\s*cat(egory|egories)?\b|age\s*grp\b|age\-group",
        r"\bgender\b|\bsex\b",
        r"\brace\b|ethnic|hispanic",
        r"\bregion\b|census",
        r"\burban(ization)?\b|rural\b",
        r"\bstate\b|^us state$|^state name$",
        r"\boverall\b|^total$"
    ]
    for pat in patterns:
        try:
            c = _pick_col(df, [pat], required=False, exclude_regex=exclude)
            if c: return c
        except: 
            pass
    return None  # overall-only

def load_observed_excel(path, sheet_name=None, overall_label="Overall"):
    xls = pd.ExcelFile(path)
    chosen = sheet_name or next(
        (s for s in xls.sheet_names if "data" in s.lower() and "final" in s.lower()),
        xls.sheet_names[0]
    )
    df = pd.read_excel(xls, sheet_name=chosen)
    df = _flatten_columns(df)

    ycol = find_year(df)
    tcol = find_aamr(df)
    gcol = find_group(df)  # may be None

    df[ycol] = pd.to_numeric(df[ycol], errors="coerce")
    df[tcol] = pd.to_numeric(df[tcol], errors="coerce")

    if gcol == tcol:  # safety
        gcol = None

    if gcol is None:
        tidy = (df[[ycol, tcol]]
                .dropna(subset=[ycol, tcol])
                .groupby(ycol, as_index=False)[tcol].mean()
                .rename(columns={ycol:"Year", tcol:"AAMR"}))
        tidy["Group"] = overall_label
        tidy = tidy[["Year","Group","AAMR"]]
    else:
        df[gcol] = df[gcol].astype(str).str.strip()
        tidy = (df[[ycol, gcol, tcol]]
                .dropna(subset=[ycol, gcol, tcol])
                .groupby([ycol, gcol], as_index=False)[tcol].mean()
                .rename(columns={ycol:"Year", gcol:"Group", tcol:"AAMR"}))

    tidy["Year"] = tidy["Year"].astype(int)
    tidy = tidy.sort_values(["Group","Year"]).reset_index(drop=True)

    print(f"Sheet: {chosen}")
    print(f"Detected → Year: '{ycol}' | AAMR: '{tcol}' | Group: '{(gcol or overall_label)}'")
    print("First rows:\n", tidy.head(6))
    return tidy

observed = load_observed_excel(FILE_PATH, sheet_name=SHEET_NAME, overall_label="Overall")


Sheet: data final
Detected → Year: 'Year' | AAMR: 'Age Adjusted Rate' | Group: 'Census Region'
First rows:
    Year                       Group   AAMR
0  1999  Census Region 1: Northeast  16.38
1  2000  Census Region 1: Northeast  15.69
2  2001  Census Region 1: Northeast  15.68
3  2002  Census Region 1: Northeast  16.20
4  2003  Census Region 1: Northeast  15.07
5  2004  Census Region 1: Northeast  14.33


In [111]:
def _sanitize_series_for_arima(y: pd.Series):
    y = y.sort_index()
    full = pd.Index(range(int(y.index.min()), int(y.index.max())+1), name=y.index.name)
    y = y.reindex(full)
    if y.isna().any():
        y = y.interpolate(limit_direction="both")
    return y

def select_arima_order(y):
    best = None
    for d in [0,1,2]:
        for p in range(0,4):
            for q in range(0,4):
                if (p,d,q) == (0,0,0): 
                    continue
                try:
                    trend = "n" if d>0 else "c"
                    res = ARIMA(y, order=(p,d,q), trend=trend,
                                enforce_stationarity=False, enforce_invertibility=False
                               ).fit(method_kwargs={"warn_convergence":False})
                    score = res.aic
                    if (best is None) or (score < best[0]):
                        best = (score, (p,d,q,trend), res)
                except Exception:
                    pass
    if best is None:
        res = ARIMA(y, order=(1,1,0), trend="n",
                    enforce_stationarity=False, enforce_invertibility=False
                   ).fit(method_kwargs={"warn_convergence":False})
        return (1,1,0,"n"), res
    return best[1], best[2]

def forecast_to(y, end_year, conf=0.95):
    last_year = int(y.index.max())
    steps = max(0, end_year - last_year)
    if steps == 0:
        raise ValueError("Observed series already extends to END_YEAR.")
    order, res = select_arima_order(y)
    fc = res.get_forecast(steps=steps)
    mean = fc.predicted_mean
    ci   = fc.conf_int(alpha=1-conf)
    lo, hi = ci.iloc[:,0], ci.iloc[:,1]
    yrs = list(range(last_year+1, end_year+1))
    out = pd.DataFrame({
        "Year": yrs,
        "Point.Forecast": mean.values,
        "Lo.95": lo.values,
        "Hi.95": hi.values,
        "Order": [f"{order[0]},{order[1]},{order[2]} ({order[3]})"]*steps
    })
    return order, out


In [112]:
def safe_name(s: str) -> str:
    return re.sub(r"[^a-zA-Z0-9_-]+", "_", str(s)).strip("_")

def fit_all_groups_and_save(observed_df, end_year, out_dir: Path, title_prefix="CKD & Stroke"):
    out_dir.mkdir(parents=True, exist_ok=True)
    groups = sorted(observed_df["Group"].unique())
    all_rows = []
    last_obs_year = int(observed_df["Year"].max())

    for g in groups:
        sub = observed_df[observed_df["Group"]==g].copy()
        sub = sub.dropna(subset=["Year","AAMR"]).sort_values("Year")
        y = pd.Series(sub["AAMR"].values, index=sub["Year"].astype(int), name="AAMR")
        y = _sanitize_series_for_arima(y)

        order, fc = forecast_to(y, end_year=end_year, conf=0.95)

        # save per-group csv (two decimals)
        csv_path = out_dir / f"{safe_name(g)}_forecast_to_{end_year}.csv"
        fc_out = fc.copy()
        for c in ["Point.Forecast","Lo.95","Hi.95"]:
            fc_out[c] = fc_out[c].round(2)
        fc_out.insert(0, "Series", g)
        fc_out.to_csv(csv_path, index=False)
        print(f"[{g}] order={order}  -> saved CSV:", csv_path)

        # keep for consolidated
        tmp = fc_out.copy(); tmp["Group"] = g
        all_rows.append(tmp)

        # per-group plot
        fig, ax = plt.subplots()
        ax.plot(y.index, y.values, marker="o", label=f"{g} (obs)")
        ln, = ax.plot(fc["Year"], fc["Point.Forecast"], linestyle="--", marker="o", label=f"{g} (fc)")
        ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.14, color=ln.get_color(), label="95% CI (fc)")
        ax.axvline(x=last_obs_year+0.5, linestyle=":", linewidth=2.0)
        ax.set_title(f"{title_prefix}: {g} — observed (≤{last_obs_year}) & ARIMA forecast ({last_obs_year+1}–{end_year})")
        ax.set_xlabel("Year"); ax.set_ylabel("AAMR (per 100,000)")
        ax.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
        style_axes(ax)
        ax.legend(ncols=2)
        plt.tight_layout()
        png_path = out_dir / f"{safe_name(g)}_timeseries_to_{end_year}.png"
        plt.savefig(png_path, dpi=300, bbox_inches="tight"); plt.close()
        print("   saved plot:", png_path)

    if all_rows:
        all_df = pd.concat(all_rows, ignore_index=True)
        all_df.to_csv(out_dir / f"ALL_GROUPS_forecasts_to_{end_year}.csv", index=False)
        print("Saved consolidated CSV:", out_dir / f"ALL_GROUPS_forecasts_to_{end_year}.csv")

fit_all_groups_and_save(observed, END_YEAR, OUTPUT_DIR, title_prefix="CKD & Stroke")


[Census Region 1: Northeast] order=(0, 2, 3, 'n')  -> saved CSV: D:\arima project\code\ckd_stroke_results\region\Census_Region_1_Northeast_forecast_to_2043.csv
   saved plot: D:\arima project\code\ckd_stroke_results\region\Census_Region_1_Northeast_timeseries_to_2043.png
[Census Region 2: Midwest] order=(0, 2, 3, 'n')  -> saved CSV: D:\arima project\code\ckd_stroke_results\region\Census_Region_2_Midwest_forecast_to_2043.csv
   saved plot: D:\arima project\code\ckd_stroke_results\region\Census_Region_2_Midwest_timeseries_to_2043.png
[Census Region 3: South] order=(0, 2, 3, 'n')  -> saved CSV: D:\arima project\code\ckd_stroke_results\region\Census_Region_3_South_forecast_to_2043.csv
   saved plot: D:\arima project\code\ckd_stroke_results\region\Census_Region_3_South_timeseries_to_2043.png
[Census Region 4: West] order=(0, 2, 3, 'n')  -> saved CSV: D:\arima project\code\ckd_stroke_results\region\Census_Region_4_West_forecast_to_2043.csv
   saved plot: D:\arima project\code\ckd_stroke_resu

In [113]:
def plot_combined(observed_df, out_dir:Path, end_year:int, title="CKD & Stroke"):
    last_obs_year = int(observed_df["Year"].max())
    groups = sorted(observed_df["Group"].unique())

    # read saved forecasts for CI
    fc_map = {}
    for g in groups:
        p = out_dir / f"{safe_name(g)}_forecast_to_{end_year}.csv"
        if p.exists():
            fc_map[g] = pd.read_csv(p)

    fig, ax = plt.subplots()
    ci_legend_added = False

    for g in groups:
        sub = observed_df[observed_df["Group"]==g].copy().sort_values("Year")
        ax.plot(sub["Year"], sub["AAMR"], marker="o", label=f"{g} (obs)")
        if g in fc_map:
            fc = fc_map[g]
            ln, = ax.plot(fc["Year"], fc["Point.Forecast"], linestyle="--", marker="o", label=f"{g} (fc)")
            # one legend entry for CI:
            if not ci_legend_added:
                ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.12, color=ln.get_color(), label="95% CI (fc)")
                ci_legend_added = True
            else:
                ax.fill_between(fc["Year"], fc["Lo.95"], fc["Hi.95"], alpha=0.12, color=ln.get_color())

    ax.axvline(x=last_obs_year+0.5, linestyle=":", linewidth=2.0)
    ax.set_title(f"{title}: observed (≤{last_obs_year}) & ARIMA forecast ({last_obs_year+1}–{end_year})")
    ax.set_xlabel("Year"); ax.set_ylabel("AAMR (per 100,000)")
    ax.yaxis.set_major_formatter(FormatStrFormatter("%.2f"))
    style_axes(ax)
    ax.legend(ncols=2)
    ax.margins(x=0.02)
    plt.tight_layout()
    out = out_dir / f"COMBINED_timeseries_to_{end_year}.png"
    plt.savefig(out, dpi=300, bbox_inches="tight"); plt.close()
    print("Saved combined plot:", out)
    
plot_combined(observed, OUTPUT_DIR, END_YEAR, title="CKD & Stroke")


Saved combined plot: D:\arima project\code\ckd_stroke_results\region\COMBINED_timeseries_to_2043.png
